# Creating an interactive gamebook

## What is a gamebook

This might a relic of the past, but in the past there were (mostly fantasy) book that were telling a story while also being interactive and letting you choose your own adventure. It was like a DnD, but for only one player.

They worked the following way:
You were reading the book like you would normally, but every couple of sentences or paragraphs, you'd have to make a choice how to continue. The choice involved two things. Each choice would lead you to a different page or paragraph of the book. Something like "If you want to go through the door, go to 43. If you want to try your luck with the staircase, go to 182". Additionally, there could be an element of luck. There, you'd roll dice and see if you manage to avoid some obstacle. Many books also include a simple class and/or abilities system, where you choose what kind of character you want to be. You might choose to be a warrior or a sorcerer. Or you might put most of the skill point to intelligence and agility, as opposed to strength. These traits would influence the results of your dice rolls. 

In [ ]:
!python -m pip install -r requirements.txt

## Getting gamebooks

Example game books are not included in this repository for licensing reasons. You have to supply your own.
These can be bought, although it might theoretically be a problem with DRM-protected books.

### Some free options

The gamebook genre is not as popular as other categories, but you CAN find a number of them onlines. Additionally, you can get your hands on some really good amateur ones that you can download for free. These are a very good start.

I got some from the [Fighting Fantasy project](http://www.ffproject.com/download.htm)

### Using a downloaded book

After you download a book, you can put it in the `gamebooks` folder. To start using it in this playbook, set its path into the `gamebook_pdf_path` variable (currently only works with PDFs).

## Reading the gamebook

We will accept gamebooks as pdf or text file. We will read the text inside and try to understand how the game works

In [ ]:
gamebook_pdf_path = 'gamebooks/PHOBIA.pdf'

In [ ]:
# Add knowledgebase from a PDF

from extract_pdf import extract_page_texts
texts = extract_page_texts(gamebook_pdf_path)
texts[:6]

### Create a LLM object

Beyond this point in the notebook, we will start using LLMs, which means we have to finally initialize DSPy and speficy some parameters

#### Local LLM

To make things ~simpler~ cheaper, let's use a locallly running LLM. Ollama is an excellent choice. Make sure it's running and it is setup with a some model.

The model we will be using here is Mistral. You are free to change it to another one that you like.

#### Retrieval

Although not strictly necessary here, we will also setup a locally running ChromaDB, in case we decide to use some embeddings

In [ ]:
import dspy

import dspy
from dspy.retrieve.chromadb_rm import ChromadbRM
from chromadb.utils.embedding_functions import DefaultEmbeddingFunction

chroma_dir = './chroma'
chroma_collection = 'book_data'

chroma_rm = ChromadbRM(
    collection_name=chroma_collection,
    persist_directory=chroma_dir,
    embedding_function=DefaultEmbeddingFunction(),
    k=3,
)

mistral_ollama = dspy.OllamaLocal(
    model='mistral',
    max_tokens=1000,
)
dspy.configure(
    lm=mistral_ollama,
    rm=chroma_rm
)


## Understanding the game rules

Most books have similar rules, but they DO usually differ a tiny bit. The most important thing is to understand if the protagonist has any stats or abilities to setup before beginning the game.

In [ ]:
# Ask an LLM to explain the game ruled based on the initial few pages

class BookRulesSignature(dspy.Signature):
    """The following context describes how to play a game. Can you summarize the rules and tell me the input parameters it has?"""

    context = dspy.InputField(desc="The first few pages of the book")
    answer = dspy.OutputField()


class BookRules(dspy.Module):
    def __init__(self, pages: list[str], num_pages=6):
        super().__init__()

        self.context = '\n'.join(pages[:num_pages])
        self.generate_answer = dspy.ChainOfThought(BookRulesSignature)
    
    def forward(self) -> dspy.Prediction:
        prediction = self.generate_answer(context=self.context)
        return dspy.Prediction(context=self.context, answer=prediction.answer)

In [ ]:
# get the rules
rule_generator = BookRules(texts)
prediction = rule_generator.forward()
prediction.answer

#### Alternative - hardcoded rules

Not all LLMs are created equal. For some, it's difficult to follow some instructions and they fail to provide good results.
This was exactly the case with Mistral and the game instructions in this case. Since this is just a demonstration playbook, we can provide the option to hardcode the game rules.

The rules you see below are the ones for a book called Phobia, which is available from the [Fighting Fantasy project](http://www.ffproject.com/download.htm).

The following game rules are ones that were generated via OpenAI's chatGPT 3.5 Turbo. So you will get better results with the pipeline above if you use OpenAI, instead of Mistral.

In [ ]:
# save the game rules
game_rules = '''
Game Summary:

Introduction:
Players assume the role of a hero with a significant weakness—a deep-seated phobia.

Rules:

Stats: Players have four stats: Strength, Luck, Skill, and Fear. These stats range from 0 to 12.
Stat Tests: Players roll two six-sided dice to test their stats. If the roll is less than or equal to the stat value, they pass; otherwise, they fail.
Fear Tests: Fear tests are handled inversely; if the roll exceeds the fear score, it's a pass. Fear increases upon encountering frightening elements but decreases if calmed.
Items: Players collect items during the adventure, each with specific rules detailed in relevant sections.
Character Creation: Allocate 24 points among Strength, Luck, and Skill (max 10 per stat). Roll one die to determine initial fear. Choose a phobia from a list provided.
Input Parameters:

Strength: Points allocated (0-10)
Luck: Points allocated (0-10)
Skill: Points allocated (0-10)
Initial Fear: Roll of one die
Phobia: Selection from the provided list (Claustrophobia, Mysophobia, Zoophobia, Acrophobia, Nyctophobia, Aquaphobia)
'''

## Create a game run

The game rules should include instructions how to generate a new chracter / game run. It probably involves setting up stats

### Game stats

It is quite usual for a gamebook to ask players to create a character using a point system. This point system has the player choose a numeric value for a number of character traits and abilities. Often times, the total amount of point is limited, so the player needs to prioritize some traits over others.

### Creating stats

We will create an LLM pipeline that takes the games rules and a player-provided string that explains in humna language what they want their character to look like.

In [ ]:
class GameRunSignature(dspy.Signature):
    """Based on the game rules in the context and the input form the player, write down all the necessary information to play the game"""

    context = dspy.InputField(desc="The rules for playing the game")
    game_parameters = dspy.InputField(desc="Should contain the information required to setup a new game")
    answer = dspy.OutputField()


class GameRun(dspy.Module):
    def __init__(self, rules: str):
        super().__init__()

        self.rules = rules
        self.generate_answer = dspy.ChainOfThought(GameRunSignature)
    
    def forward(self, input_parameters: str) -> dspy.Prediction:
        prediction = self.generate_answer(context=self.rules, game_parameters=input_parameters)
        return dspy.Prediction(context=self.rules, answer=prediction.answer)

### Testing the character creation

In [ ]:
user_answer_stats = "I will have Claustrophobia, my Fear is 4, my Strength is 7, Luck 9 and Skill 8"
game_run_parameters = GameRun(game_rules)
game_run = game_run_parameters.forward(user_answer_stats)
game_run.answer

## Parsing the book sections

The main sections of game books are the snippets containing elements of the story. They are usually numbered and easy to find. However, each book might use a different convention for denoting them. A safer bet is to ask an LLM to do the splitting. The problem is that a book can easily be longer than the maximum context length that can be handled at once, so this needs to happen in chunks.

We will iterate the book piece by piece and collect all snippets

In [ ]:
# a page is too little, and it could be that a snippet spans beyond a single page.
# we will use a semantic text splitter and chunks that are not page-bound

# NOTE: this does not scale well - everything will be in-memory, several times
all_text = '\n\n'.join(texts[5:])

from semantic_text_splitter import TextSplitter

# Maximum number of characters in a chunk
max_characters = 2000
# Optionally can also have the splitter not trim whitespace for you
splitter = TextSplitter()

chunks = splitter.chunks(all_text, max_characters)

In [ ]:
class SnippetSplitSignature(dspy.Signature):
    """The provided context contains several numbered sections. The border between sections should be the different item numbers, serving as titles of the sections. Return a JSON array containing an object for each numbered paragraph with properties - the item number and its contents (don't change the contents)"""

    context = dspy.InputField(desc="Text containing several numbered sections")
    answer = dspy.OutputField()


class SnippetSplitRun(dspy.Module):
    def __init__(self, paragrpahs: str):
        super().__init__()

        self.paragrpahs = paragrpahs
        self.generate_answer = dspy.ChainOfThought(SnippetSplitSignature)
    
    def forward(self) -> dspy.Prediction:
        prediction = self.generate_answer(context=self.paragrpahs)
        return dspy.Prediction(context=self.paragrpahs, answer=prediction.answer)

In [ ]:
for chunk in chunks:
    split_run = SnippetSplitRun(chunk)
    snippet_predition = split_run.forward()
    print(snippet_predition.answer)

### Hardcoded snippets

Much like the game rules, some LLMs fail at properly splitting the book into game snippets. Since the split is usually not that difficult to do using a pure algorithmic approach, we will give that option here.

Ofc, this way is not very flexible because it assumes the split between snippets is a number, followed by a line break.

Feel free to try the split with a more powerful model, but do keep in mind this algorithm goes through the whole book chunk by chunk, which typically means a lot of tokens used, and therefore, higher costs.

## Going through game steps

Now this is going to the main part of the game. Based on the following input:
* The current section in the game (a number if a paragraph in the book)
* The Game instructions
* The game run we generated (the set of stats)
* Input string from the player, where they possibly make a choice how to continue

We will create a new LLM pipeline that takes these inputs and determines how to continue the adventure. The output prediction of that pipeline will be one of the following:
* The number of the next section to go to
* Indication that the game is over

For more complex steps, it could also involve:
* Changing the game run (adding an item, changing a stat)

In [ ]:
import dspy

class GameStepSignature(dspy.Signature):
    """You are trying to determine the next step in following a gamebook. Based on the game instructions, last game step, game setup and the player's desired action, determine what happens next"""

    game_instructions = dspy.InputField(desc="Rules of the game")
    last_game_step = dspy.InputField(desc="describes what just happened in the game story")
    game_setup = dspy.InputField(desc="Details of the current game run - player state and stats")
    player_action = dspy.InputField(desc="The decision of the player how to continue")
    next_step = dspy.OutputField(desc="Should be one of the following: number of the next step to go to, game_failed or game_finished")
    step_number = dspy.OutputField(desc="Should be a valid single number only - the next step to go to")


class GameStepRun(dspy.Module):
    def __init__(self, instructions: str, step: str, setup: str, player_action: str):
        super().__init__()

        self.instructions = instructions
        self.step = step
        self.setup = setup
        self.player_action = player_action
        self.generate_answer = dspy.ChainOfThought(GameStepSignature)
    
    def forward(self) -> dspy.Prediction:
        prediction = self.generate_answer(
            game_instructions=self.instructions,
            last_game_step=self.step,
            game_setup=self.setup,
            player_action=self.player_action
        )
        context = prediction.rationale if hasattr(prediction, 'rationale') else self.step
        res = [int(i) for i in prediction.step_number.split() if i.isdigit()]
        step = res[0] if len(res) > 0 else ''
        return dspy.Prediction(context=context, answer=prediction.next_step, step=step)

In [ ]:
current_step_num = 1
stop_words = ['game_failed', 'game_finished']

while True:
    current_snippet = game_snippets[current_step_num - 1][1]
    print(current_snippet)
    player_answer = input('How would you like to continue: ')
    step_run = GameStepRun(game_rules, current_snippet, game_run.answer, player_answer)
    run_prediction = step_run.forward()
    print(run_prediction)
    print(run_prediction.answer)
    try:
        numeric_answer = int(run_prediction.step)
    except ValueError:
        numeric_answer = 0
    if numeric_answer != 0 and numeric_answer < len(game_snippets):
        current_step_num = numeric_answer
    if run_prediction.answer in stop_words:
        # It might not be immediately apparent why the game failed. Write the module's reasoning
        print(run_prediction.context)
        break;